In [1]:
import pandas
customer = pandas.read_excel('data/TenMinDataCustomerEndpoints.xlsx')
print(customer.columns)
print(customer.shape)
customer = customer[['Substation', 'datetime', 't_date', 't_time', 'P_GEN', 'Q_GEN', 'S_GEN']]

Index(['Substation', 'datetime', 't_date', 't_time', 'd_y', 'd_m', 'd_d',
       'd_w', 't_h', 't_m', 'V', 'V_Filtered', 'I_GEN', 'I_GEN_Filtered',
       'I_IMPORT', 'I_IMPORT_Filtered', 'P_GEN', 'P_IMPORT', 'Q_GEN',
       'Q_IMPORT', 'S_GEN', 'S_IMPORT', 'thdV', 'thdV_Filtered', 'thdI_GEN',
       'thdI_IMPORT', 'f', 'Substation_VA_Filtered', 'Substation_VB_Filtered',
       'Substation_VC_Filtered', 'Substation_thdVA_Filtered',
       'Substation_thdVB_Filtered', 'Substation_thdVC_Filtered', 'VA_Rise',
       'VB_Rise', 'VC_Rise'],
      dtype='object')
(131681, 36)


In [2]:
customer.head()

,Substation,datetime,t_date,t_time,P_GEN,Q_GEN,S_GEN
0,Forest Road,2014-06-10 02:10:00,2014-06-10,02:10:00,0.002,0.089,-0.214
1,Forest Road,2014-06-10 02:20:00,2014-06-10,02:20:00,0.002,0.089,-0.212
2,Forest Road,2014-06-10 02:30:00,2014-06-10,02:30:00,0.002,0.089,-0.211
3,Forest Road,2014-06-10 02:40:00,2014-06-10,02:40:00,0.002,0.089,-0.212
4,Forest Road,2014-06-10 02:50:00,2014-06-10,02:50:00,0.002,0.089,-0.215


#### Removing Suffolk Road & YMCA

In [3]:
print(customer.shape)
customer.Substation.value_counts()

(131681, 7)


Suffolk Road        23428
YMCA                23425
Bancroft Close      23061
Alverston Close     22660
Maple Drive East    22500
Forest Road         16607
Name: Substation, dtype: int64

In [4]:
customer = customer[~((customer.Substation == 'YMCA') | (customer.Substation == 'Suffolk Road'))]
print(customer.shape)
customer.Substation.value_counts()

(84828, 7)


Bancroft Close      23061
Alverston Close     22660
Maple Drive East    22500
Forest Road         16607
Name: Substation, dtype: int64

#### Cleaning Alverston Close

In [5]:
customer[customer.Substation == 'Alverston Close']['P_GEN'].min()

-1680.0

In [6]:
import plotly.express as px
fig = px.scatter(x=customer[customer.Substation == 'Alverston Close'].datetime, y=customer[customer.Substation == 'Alverston Close']['P_GEN'])
fig.show()

In [7]:
def f(row):
    if ((row['Substation'] == 'Alverston Close') and (row['P_GEN'] < -1300)):
        val = abs(row['P_GEN'])
    else:
        val = row['P_GEN']
    return val

customer['P_GEN'] = customer.apply(f, axis=1)

In [8]:
fig = px.scatter(x=customer[customer.Substation == 'Alverston Close'].datetime, y=customer[customer.Substation == 'Alverston Close']['P_GEN'])
fig.show()

In [9]:
weather = pandas.read_excel('data/WeatherData2014-11-30.xlsx')
weather = weather[['Site', 'Date', 'Time', 'TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']]
weather.head()

,Site,Date,Time,TempOut,SolarRad,SolarEnergy,HiSolarRad
0,YMCA,2014-01-07,14:30:00,10.6,32,1.38,44
1,YMCA,2014-01-07,15:00:00,10.7,20,0.86,26
2,YMCA,2014-01-07,15:30:00,10.7,22,0.95,32
3,YMCA,2014-01-07,16:00:00,10.4,8,0.34,14
4,YMCA,2014-01-07,16:30:00,10.3,0,0,0


In [10]:
import numpy as np
weather['wrong_times'] = weather['Time'].apply(lambda x: x if len(str(x))>8 else np.nan)

In [11]:
weather['wrong_times'].value_counts(dropna=False)

NaT           68040
1899-12-30     1440
Name: wrong_times, dtype: int64

In [12]:
weather[~weather.wrong_times.isna()][['Site', 'Date','Time']]

,Site,Date,Time
19,YMCA,2014-01-08,1899-12-30 00:00:00
67,YMCA,2014-01-09,1899-12-30 00:00:00
115,YMCA,2014-01-10,1899-12-30 00:00:00
163,YMCA,2014-01-11,1899-12-30 00:00:00
211,YMCA,2014-01-12,1899-12-30 00:00:00
...,...,...,...
69247,Easthill Road,2014-08-01,1899-12-30 00:00:00
69295,Easthill Road,2014-08-02,1899-12-30 00:00:00
69343,Easthill Road,2014-08-03,1899-12-30 00:00:00
69391,Easthill Road,2014-08-04,1899-12-30 00:00:00


In [13]:
weather.Date.value_counts(dropna=False)

2014-07-12    240
2014-04-21    240
2014-06-23    240
2014-05-24    240
2014-06-19    240
             ... 
2014-11-25     48
2013-12-03     48
2014-11-29     48
2014-11-30     40
2013-11-25     11
Name: Date, Length: 371, dtype: int64

In [14]:
weather[~weather.wrong_times.isna()][['Site', 'Date','Time']].Site.value_counts(dropna=False)

YMCA                327
Maple Drive East    314
Forest Road         305
Elm Crescent        290
Easthill Road       204
Name: Site, dtype: int64

#### Removing wrong times

In [15]:
weather['Time'] = weather['Time'].apply(lambda x: np.nan if len(str(x))>8 else x)
weather['Time'].isna().sum()
weather.drop('wrong_times', axis=1, inplace=True)

In [16]:
weather.dropna(subset = ['Time'], inplace=True)
print(weather['Time'].isna().sum())
print(weather.shape)
weather.head()

0
(68040, 7)


,Site,Date,Time,TempOut,SolarRad,SolarEnergy,HiSolarRad
0,YMCA,2014-01-07,14:30:00,10.6,32,1.38,44
1,YMCA,2014-01-07,15:00:00,10.7,20,0.86,26
2,YMCA,2014-01-07,15:30:00,10.7,22,0.95,32
3,YMCA,2014-01-07,16:00:00,10.4,8,0.34,14
4,YMCA,2014-01-07,16:30:00,10.3,0,0,0


In [17]:
weather['datetime'] = weather['Date'].astype(str) + ' ' + weather['Time'].astype(str)
weather['datetime'] = pandas.to_datetime(weather.datetime, format='%Y-%m-%d %H:%M:%S').astype(np.datetime64)
print(weather.shape)
weather.head()

(68040, 8)


,Site,Date,Time,TempOut,SolarRad,SolarEnergy,HiSolarRad,datetime
0,YMCA,2014-01-07,14:30:00,10.6,32,1.38,44,2014-01-07 14:30:00
1,YMCA,2014-01-07,15:00:00,10.7,20,0.86,26,2014-01-07 15:00:00
2,YMCA,2014-01-07,15:30:00,10.7,22,0.95,32,2014-01-07 15:30:00
3,YMCA,2014-01-07,16:00:00,10.4,8,0.34,14,2014-01-07 16:00:00
4,YMCA,2014-01-07,16:30:00,10.3,0,0,0,2014-01-07 16:30:00


#### Summer period selection for both customer and weather (21 June - 23 Sept)

In [18]:
print('Customer:')
print(customer.datetime.min())
print(customer.datetime.max())
print('Weather:')
print(weather.datetime.min())
print(weather.datetime.max())

Customer:
2014-06-10 02:10:00
2014-11-17 05:30:00
Weather:
2013-11-25 18:30:00
2014-11-30 19:30:00


In [19]:
customer = customer[(customer.datetime > '2014-06-21 00:20:00') & (customer.datetime < '2014-09-23 23:40:00')]
weather = weather[(weather.datetime > '2014-06-21') & (weather.datetime < '2014-09-24')]

In [20]:
print('Customer:')
print(customer.datetime.min())
print(customer.datetime.max())
print('Weather:')
print(weather.datetime.min())
print(weather.datetime.max())

Customer:
2014-06-21 00:30:00
2014-09-23 23:30:00
Weather:
2014-06-21 00:30:00
2014-09-23 23:30:00


In [21]:
print(customer.shape)
print(weather.shape)

(54699, 7)
(17779, 8)


In [22]:
customer.reset_index(inplace=True, drop=True)
weather.reset_index(inplace=True, drop=True)

#### Removing date and time, keeping datetime

In [23]:
customer.drop(['t_date', 't_time'], inplace=True, axis=1)
weather.drop(['Date', 'Time'], inplace=True, axis=1)

#### Replacing --- with NaN in weather table

In [24]:
import numpy as np
weather['TempOut']  = weather['TempOut'].apply(lambda x: np.nan if x=='---' else x)
weather['SolarRad']  = weather['SolarRad'].apply(lambda x: np.nan if x=='---' else x)
weather['SolarEnergy']  = weather['SolarEnergy'].apply(lambda x: np.nan if x=='---' else x)
weather['HiSolarRad']  = weather['HiSolarRad'].apply(lambda x: np.nan if x=='---' else x)
weather

,Site,TempOut,SolarRad,SolarEnergy,HiSolarRad,datetime
0,YMCA,12.6,0.0,0.0,0.0,2014-06-21 00:30:00
1,YMCA,12.3,0.0,0.0,0.0,2014-06-21 01:00:00
2,YMCA,11.4,0.0,0.0,0.0,2014-06-21 01:30:00
3,YMCA,11.7,0.0,0.0,0.0,2014-06-21 02:00:00
4,YMCA,10.6,0.0,0.0,0.0,2014-06-21 02:30:00
...,...,...,...,...,...,...
17774,Easthill Road,NaN,NaN,NaN,NaN,2014-08-05 18:00:00
17775,Easthill Road,NaN,NaN,NaN,NaN,2014-08-05 18:30:00
17776,Easthill Road,NaN,NaN,NaN,NaN,2014-08-05 19:00:00
17777,Easthill Road,NaN,NaN,NaN,NaN,2014-08-05 19:30:00


In [25]:
customer = customer.sort_values(by=['datetime'])
weather = weather.sort_values(by=['datetime'])

In [26]:
from pyarrow import feather
feather.write_feather(weather, 'data/weather_cleansed.feather')

In [27]:
feather.write_feather(customer, 'data/customer_cleansed.feather')

#### Dash 1

In [29]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

customer_feather = 'data/customer.feather'
weather_feather = 'data/weather.feather'

customer =  pandas.read_feather(customer_feather)
weather = pandas.read_feather(weather_feather)

subs = customer['Substation'].value_counts(dropna=False).keys().tolist()
sites = weather['Site'].value_counts(dropna=False).keys().tolist()

# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

colors = {
    'background': '#FFFFFF',
    'text': '#091D58'
}

app.layout = html.Div(style={'backgroundColor': colors['background']},
    children=[
        html.Div([
            html.H1(
                children="Pv data visualization",
                style={
                    'textAlign': 'center',
                    'color': colors['text']
                    }
                ),
            html.Div(
                children='Selection (from TenMinDataCustomerEndpoints.xlsx):',
                style={
                    'color': colors['text']
                    }
                ),
            dcc.Dropdown(
                id="ticker1",
                options=[{"label": x, "value": x} 
                    for x in ['P_GEN', 'Q_GEN', 'S_GEN']],
                value='P_GEN',
                clearable=False,
                style={
                    'color': colors['text']
                    }
                ),
            dcc.Graph(id='graph1')
        ]),
        html.Div([
            html.Div(
                children='Selection (from WeatherData2014-11-30.xlsx):',
                style={
                    'color': colors['text']
                    }
                ),
            dcc.Dropdown(
                id="ticker2",
                options=[{"label": x, "value": x} 
                    for x in ['TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']],
                value='SolarRad',
                clearable=False,
                style={
                    'color': colors['text']
                    }
                ),
            dcc.Graph(id='graph2')
        ])
])

#graph1
# Define callback to update graph
@app.callback(
    Output('graph1', 'figure'),
    [Input("ticker1", "value")]
)

def update_figure(ticker1):

    fig = make_subplots(6, 1, y_title = ticker1,
    subplot_titles = tuple(s for s in subs),
    shared_xaxes=True)

    s=0
    for i in range(1, 7):
        fig.add_trace(go.Scattergl(x=customer[customer.Substation == subs[s]].datetime, 
        y=customer[customer.Substation == subs[s]][ticker1], name=subs[s]), i, 1)
        s+=1

    # style all the traces
    fig.update_traces(
        line={"width": 0.5},
        marker={"size": 3},
        mode="lines+markers",
        showlegend=True
    )

    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        dragmode="zoom", template="plotly_white", legend_orientation="v", xaxis6_rangeslider_visible=True,
        xaxis6_rangeslider_thickness=0.05, xaxis6_rangeslider_bgcolor=colors['text'] , height=1000,
        xaxis1=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                    label="1d",
                    step="day",
                    stepmode="backward"),
                    dict(count=1,
                    label="1m",
                    step="month",
                    stepmode="backward"),
                    dict(count=6,
                    label="6m",
                    step="month",
                    stepmode="backward"),
                    dict(step="all")
                ])
            )
        ))

    return fig

#graph2
# Define callback to update graph
@app.callback(
    Output('graph2', 'figure'),
    [Input("ticker2", "value")]
)

def update_figure(ticker2):

    fig = make_subplots(5, 1, y_title = ticker2,
    subplot_titles = tuple(s for s in sites),
    shared_xaxes=True)

    s=0
    for i in range(1, 6):
        fig.add_trace(go.Scattergl(x=weather[weather.Site == sites[s]].Datetime, 
        y=weather[weather.Site == sites[s]][ticker2], name=sites[s]), i, 1)
        s+=1

    # style all the traces
    fig.update_traces(
        line={"width": 0.5},
        marker={"size": 3},
        mode="lines+markers",
        showlegend=True
    )

    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        dragmode="zoom", template="plotly_white", legend_orientation="v", xaxis5_rangeslider_visible=True,
        xaxis5_rangeslider_thickness=0.05, xaxis5_rangeslider_bgcolor=colors['text'] , height=1000,
        xaxis1=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                    label="1d",
                    step="day",
                    stepmode="backward"),
                    dict(count=1,
                    label="1m",
                    step="month",
                    stepmode="backward"),
                    dict(count=6,
                    label="6m",
                    step="month",
                    stepmode="backward"),
                    dict(step="all")
                ])
            )
        ))

    return fig               

# Run app and display result inline in the notebook
app.run_server(mode='external', port=8124, debug=True)

Dash app running on http://127.0.0.1:8124/


#### Dash 2

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
from dash.dependencies import Input, Output
import numpy as np
import pandas
import json

button1_state = 0
button2_state = 0

customer_feather = 'data/customer_cleansed.feather'
weather_feather = 'data/weather_cleansed.feather'

customer =  pandas.read_feather(customer_feather)
weather = pandas.read_feather(weather_feather)

# mapbox_token = open(".mapbox_token").read()

with open('data/locations.txt') as json_file:
    locations = json.load(json_file)
       
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets, prevent_initial_callbacks=True)

colors = {
    'background': '#FFFFFF',
    'text': '#091D58',
    'text_sub': '#07AD93',
    'text_site': '#430B95'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children = [
    html.Div(children=[

        html.H1('Pv data visualization', style={'textAlign': 'center', 'color': colors['text']}),

        #---------------------
        html.Div([
            html.Div([

                html.Div([
                    html.Div(children = [
                        html.H3('Substation:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_sub'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='subs_ticker',
                            options=[
                                {'label': i, 'value': i} for i in customer.Substation.unique()
                                ], multi=True,
                            value=[], 
                            style={'color': colors['text']},
                            placeholder="Select a substation",
                        )
                    ], style={"width": "40%"}, className="six columns"),

                    html.Div([
                        html.H3('Signal:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_sub'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='signal1_ticker',
                            options=[
                                {'label': i, 'value': i} for i in ['P_GEN', 'Q_GEN', 'S_GEN']
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']}
                        ),
                    ], style={"width": "40%"}, className="six columns"),

                ], className="row"),

                html.Div([
                    html.Div([
                        html.H3('Weather site:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_site'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='sites_ticker',
                            options=[
                                {'label': i, 'value': i} for i in weather.Site.unique()
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']},
                            placeholder="Select a weather site"
                        ),
                    ], style={"width": "40%"}, className="six columns"),
                    html.Div([
                        html.H3('Signal:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_site'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='signal2_ticker',
                            options=[
                                {'label': i, 'value': i} for i in ['TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']},
                        ),
                    ], style={"width": "40%"}, className="six columns")
                ], className="row"),

                html.Div([
                    html.H3('Ratios:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                    dcc.Dropdown(
                        id='ratio_ticker',
                        options=[
                            {'label': i, 'value': i} for i in [
                                'P_GEN/SolarRad', 'P_GEN/SolarEnergy',
                                'Q_GEN/SolarRad', 'Q_GEN/SolarEnergy',
                                'S_GEN/SolarRad', 'S_GEN/SolarEnergy']
                            ], multi=True,
                        value=[],
                        style={'color': colors['text']},
                    ),
                ], style={"width": "85%"}),

                html.H3('Download displayed traces:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                html.Div([
                    html.Div([
                        html.Button("Substations Traces", id="btn1"), Download(id="download1")
                    ], className="six columns", style={"width": "20%", 'color': colors['text']}),
                    html.Div([
                        html.Button("Weather Traces", id="btn2"), Download(id="download2")
                    ], className="six columns", style={"width": "20%", 'color': colors['text'], 'padding-left':'25%'})
                ], className="row"),

            ], className="six columns"),
            #------------------

            html.Div([
                dcc.Graph(id='map')
            ], className="six columns"),
        ], className="row", style={'marginTop': '5em','padding-left':'10%', 'padding-right':'15%', 'verical-align': 'center'}),
        #---------------

        html.Div([
            dcc.Graph(id='graph')
        ], style={'padding-left':'1%', 'padding-right':'1%'}),
        ]),

    html.Div(children=[

        html.Div([

            # two dropdown menu
            html.Div([

                # substation dropdown
                html.Div([
                    html.H1('Correlations', style={'textAlign': 'left', 'fontSize': 20, 'color': colors['text']}),
                    html.H3('Substation:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                    dcc.Dropdown(
                        id='subs_ticker_corr',
                        options=[
                            {'label': i, 'value': i} for i in customer.Substation.unique()
                            ], multi=False, clearable=False,
                        value='Forest Road', 
                        style={'color': colors['text']}
                    ),
                ]),

                # weather site dropdown
                html.Div([
                    html.H3('Weather site:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                    dcc.Dropdown(
                        id='sites_ticker_corr',
                        options=[
                            {'label': i, 'value': i} for i in weather.Site.unique()
                            ], multi=False, clearable=False,
                        value='Forest Road', 
                        style={'color': colors['text']}
                    ),
                ]),

            ], className='three columns'),

            # first image 
            html.Div(
                children = [
                    dcc.Graph(id='corr_c')
            ], className='three columns'),

            # second image
            html.Div(
                children = [
                    dcc.Graph(id='corr_w')
            ], className='three columns'),

            # third image 
            html.Div(
                children = [
                    dcc.Graph(id='corr_tot')
                ], className = 'three columns'),

        ], className='row', style = {'padding-left':'5%', 'padding-right':'10%'})

    ])

])

###########
@app.callback(Output("download1", "data"),
[Input("btn1", "n_clicks"),
Input('subs_ticker', 'value'),
Input('signal1_ticker', 'value')])
def func(n_clicks, selected_subs, selected_sig1):
    global button1_state
    if n_clicks is not None:
        if n_clicks > button1_state:
            selected_sig1.append('Substation')
            selected_sig1.append('datetime')
            sel_customer = customer[selected_sig1]
            sel_customer = sel_customer[sel_customer['Substation'].isin(selected_subs)]

            button1_state += 1

            return send_data_frame(sel_customer.to_excel, "substations.xls", index=False)


@app.callback(Output("download2", "data"),
[Input("btn2", "n_clicks"),
Input('sites_ticker', 'value'),
Input('signal2_ticker', 'value')])
def func(n_clicks, selected_sites, selected_sig2):
    global button2_state
    if n_clicks is not None:
        if n_clicks > button2_state:
            selected_sig2.append('Site')
            selected_sig2.append('datetime')
            sel_weather = weather[selected_sig2]
            sel_weather = sel_weather[sel_weather['Site'].isin(selected_sites)]

            button2_state += 1

            return send_data_frame(sel_weather.to_excel, "weather.xls", index=False)
################

@app.callback(Output('map', 'figure'),
[Input('subs_ticker', 'value'),
Input('sites_ticker', 'value')])
def update_map(selected_subs, selected_sites):

    fig = go.Figure()

    lat = []
    lon = []
    text = []
    for sub in selected_subs:
        lat.append(locations[sub]['lat'])
        lon.append(locations[sub]['lon'])
        text.append(sub)
    for site in selected_sites:
        lat.append(locations[site]['lat'])
        lon.append(locations[site]['lon'])
        text.append(site)    

    fig = go.Figure(go.Scattermapbox(
            lat=lat,
            lon=lon,
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=14
            ),
            text=text,
        ))

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            #accesstoken=mapbox_token,
            bearing=0,
            style = 'open-street-map',
            center=go.layout.mapbox.Center(
                lat=51.89,
                lon=0.93
            ),
            pitch=0,
            zoom=5
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        width=500, 
        height=400
    )

    return fig

@app.callback(Output('graph', 'figure'),
[Input('subs_ticker', 'value'),
Input('signal1_ticker', 'value'),
Input('sites_ticker', 'value'),
Input('signal2_ticker', 'value'),
Input('ratio_ticker', 'value')])
def update_graph(selected_subs, selected_sig1, selected_sites, selected_sig2, selected_ratios):
    fig = go.Figure()

    for x in selected_subs:
        for y in selected_sig1:
            fig.add_trace(go.Scattergl(x=customer[customer.Substation == x].datetime, y=customer[customer.Substation == x][y],
            name = str(x) + ' ' + str(y)))

    for x in selected_sites:
        for y in selected_sig2:
            fig.add_trace(go.Scattergl(x=weather[weather.Site == x].datetime, y=weather[weather.Site == x][y],
            name = str(x) + ' ' + str(y)))

    for sub in selected_subs:
        for site in selected_sites:
            for ratio in selected_ratios: # P_GEN/SolarRad
                weather_upsampled = weather[weather.Site == site].set_index('datetime').resample('10T').asfreq().reset_index()
                denom = weather_upsampled[ratio[6:]]
                num = customer[customer.Substation == sub][ratio[0:5]]
                fig.add_trace(go.Scattergl(x=customer[customer.Substation == sub].datetime, y=num.divide(denom).replace(np.inf, 0),
                name = str(sub) + ' ' + str(ratio[0:5]) + '\\' + str(site) + ' ' + str(ratio[6:])))

    fig.update_traces(
        line={"width": 0.5},
        marker={"size": 3},
        mode="lines+markers",
        showlegend=True
    )

    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        dragmode="zoom", template="plotly_white", legend_orientation="v", xaxis_rangeslider_visible=True,
        xaxis_rangeslider_thickness=0.1, xaxis_rangeslider_bgcolor=colors['text'],
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                    label="1d",
                    step="day",
                    stepmode="backward"),
                    dict(count=1,
                    label="1m",
                    step="month",
                    stepmode="backward"),
                    dict(step="all")
                ])
            )
        ))
    
    return fig

@app.callback(
    Output('corr_c', 'figure'),
    Input('subs_ticker_corr', 'value')
    )
def update_graph_c(selected_sub):

    corr_c = customer[customer.Substation == selected_sub][['P_GEN', 'Q_GEN', 'S_GEN']].corr()
    #corr_w = weather[weather.Site == selected_site][['TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']].corr()

    fig = go.Figure(go.Heatmap(
                        z=corr_c,
                        x=corr_c.columns,
                    y=corr_c.columns,
                    hoverongaps = False,
                    colorbar = dict(x=1.2)))

    fig.update_layout(
        title={
            'text': "Customer Substation",
            'y':0.15,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'middle'})

    fig.update_layout(height=300, width=300)
    fig.update_xaxes(side="top")

    return fig

@app.callback(
    Output('corr_w', 'figure'),
    Input('sites_ticker_corr', 'value')
    )
def update_graph_w(selected_site):

    corr_w = weather[weather.Site == selected_site][['TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']].corr()

    fig = go.Figure(go.Heatmap(
                        z=corr_w,
                        x=corr_w.columns,
                    y=corr_w.columns,
                    hoverongaps = False,
                    colorbar = dict(x=1.2)))
    fig.update_layout(
        title={
            'text': "Weather Site",
            'y':0.15,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'middle'})

    fig.update_layout(height=300, width=300)
    fig.update_xaxes(side="top")

    return fig

@app.callback(
    Output('corr_tot', 'figure'),
    [
        Input('subs_ticker_corr', 'value'),
        Input('sites_ticker_corr', 'value')
    ]
)
def update_graph_tot(selected_sub, selected_site):

    w = weather[weather.Site == selected_sub].set_index('datetime').resample('10T').asfreq().reset_index()
    c = customer[customer.Substation == selected_site]

    # Setting the timestamp as the index
    c = c.set_index('datetime')
    w = w.set_index('datetime')

    # perform a join and that's it
    joined = c.join(w, how='outer')

    corr = joined[['P_GEN', 'Q_GEN', 'S_GEN', 'TempOut', 'SolarRad',
        'SolarEnergy', 'HiSolarRad']].corr()

    fig = go.Figure(data=go.Heatmap(
                        z=corr,
                        x=corr.columns,
                    y=corr.columns,
                    hoverongaps = False))

    fig.update_layout(
        title={
            'text': "Correlation for the joined datasets",
            'y':0.15,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'middle'})

    fig.update_layout(width = 300, height = 300)
    fig.update_xaxes(side="top")

    return fig

# Run app and display result inline in the notebook
app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:8050/


#### Dash 3

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash_extensions import Download
from dash_extensions.snippets import send_data_frame
from dash.dependencies import Input, Output
import numpy as np
import pandas
import json

button1_state = 0
button2_state = 0

customer_feather = 'data/customer_cleansed.feather'
weather_feather = 'data/weather_cleansed.feather'

customer =  pandas.read_feather(customer_feather)
weather = pandas.read_feather(weather_feather)

# mapbox_token = open(".mapbox_token").read()

with open('data/locations.txt') as json_file:
    locations = json.load(json_file)
       
# Build App
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets, prevent_initial_callbacks=True)

colors = {
    'background': '#FFFFFF',
    'text': '#091D58',
    'text_sub': '#07AD93',
    'text_site': '#430B95'
}

app.layout = html.Div(style={'backgroundColor': colors['background']}, children = [
    html.Div(children=[

        html.H1('Pv data visualization', style={'textAlign': 'center', 'color': colors['text']}),

        #---------------------
        html.Div([
            html.Div([

                html.Div([
                    html.Div(children = [
                        html.H3('Substation:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_sub'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='subs_ticker',
                            options=[
                                {'label': i, 'value': i} for i in customer.Substation.unique()
                                ], multi=True,
                            value=[], 
                            style={'color': colors['text']},
                            placeholder="Select a substation",
                        )
                    ], style={"width": "40%"}, className="six columns"),

                    html.Div([
                        html.H3('Signal:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_sub'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='signal1_ticker',
                            options=[
                                {'label': i, 'value': i} for i in ['P_GEN', 'Q_GEN', 'S_GEN']
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']}
                        ),
                    ], style={"width": "40%"}, className="six columns"),

                ], className="row"),

                html.Div([
                    html.Div([
                        html.H3('Weather site:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_site'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='sites_ticker',
                            options=[
                                {'label': i, 'value': i} for i in weather.Site.unique()
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']},
                            placeholder="Select a weather site"
                        ),
                    ], style={"width": "40%"}, className="six columns"),
                    html.Div([
                        html.H3('Signal:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text_site'], 'font-weight': 'bold'}),
                        dcc.Dropdown(
                            id='signal2_ticker',
                            options=[
                                {'label': i, 'value': i} for i in ['TempOut', 'SolarRad', 'SolarEnergy', 'HiSolarRad']
                                ], multi=True,
                            value=[],
                            style={'color': colors['text']},
                        ),
                    ], style={"width": "40%"}, className="six columns")
                ], className="row"),

                html.Div([
                    html.H3('Ratios:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                    dcc.Dropdown(
                        id='ratio_ticker',
                        options=[
                            {'label': i, 'value': i} for i in [
                                'P_GEN/SolarRad', 'P_GEN/SolarEnergy',
                                'Q_GEN/SolarRad', 'Q_GEN/SolarEnergy',
                                'S_GEN/SolarRad', 'S_GEN/SolarEnergy']
                            ], multi=True,
                        value=[],
                        style={'color': colors['text']},
                    ),
                ], style={"width": "85%"}),

                html.H3('Download displayed traces:', style={'paddingRight': '30px', 'fontSize': 18, 'color': colors['text']}),
                html.Div([
                    html.Div([
                        html.Button("Substations Traces", id="btn1"), Download(id="download1")
                    ], className="six columns", style={"width": "20%", 'color': colors['text']}),
                    html.Div([
                        html.Button("Weather Traces", id="btn2"), Download(id="download2")
                    ], className="six columns", style={"width": "20%", 'color': colors['text'], 'padding-left':'25%'})
                ], className="row"),

            ], className="six columns"),
            #------------------

            html.Div([
                dcc.Graph(id='map')
            ], className="six columns"),
        ], className="row", style={'marginTop': '5em','padding-left':'10%', 'padding-right':'15%', 'verical-align': 'center'}),
        #---------------

        html.Div([
            dcc.Graph(id='graph')
        ], style={'padding-left':'1%', 'padding-right':'1%'}),
        ])

])

###########
@app.callback(Output("download1", "data"),
[Input("btn1", "n_clicks"),
Input('subs_ticker', 'value'),
Input('signal1_ticker', 'value')])
def func(n_clicks, selected_subs, selected_sig1):
    global button1_state
    if n_clicks is not None:
        if n_clicks > button1_state:
            selected_sig1.append('Substation')
            selected_sig1.append('datetime')
            sel_customer = customer[selected_sig1]
            sel_customer = sel_customer[sel_customer['Substation'].isin(selected_subs)]

            button1_state += 1

            return send_data_frame(sel_customer.to_excel, "substations.xls", index=False)


@app.callback(Output("download2", "data"),
[Input("btn2", "n_clicks"),
Input('sites_ticker', 'value'),
Input('signal2_ticker', 'value')])
def func(n_clicks, selected_sites, selected_sig2):
    global button2_state
    if n_clicks is not None:
        if n_clicks > button2_state:
            selected_sig2.append('Site')
            selected_sig2.append('datetime')
            sel_weather = weather[selected_sig2]
            sel_weather = sel_weather[sel_weather['Site'].isin(selected_sites)]

            button2_state += 1

            return send_data_frame(sel_weather.to_excel, "weather.xls", index=False)
################

@app.callback(Output('map', 'figure'),
[Input('subs_ticker', 'value'),
Input('sites_ticker', 'value')])
def update_map(selected_subs, selected_sites):

    fig = go.Figure()

    lat = []
    lon = []
    text = []
    for sub in selected_subs:
        lat.append(locations[sub]['lat'])
        lon.append(locations[sub]['lon'])
        text.append(sub)
    for site in selected_sites:
        lat.append(locations[site]['lat'])
        lon.append(locations[site]['lon'])
        text.append(site)    

    fig = go.Figure(go.Scattermapbox(
            lat=lat,
            lon=lon,
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=14
            ),
            text=text,
        ))

    fig.update_layout(
        hovermode='closest',
        mapbox=dict(
            #accesstoken=mapbox_token,
            bearing=0,
            style = 'open-street-map',
            center=go.layout.mapbox.Center(
                lat=51.89,
                lon=0.93
            ),
            pitch=0,
            zoom=5
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        width=500, 
        height=400
    )

    return fig

@app.callback(Output('graph', 'figure'),
[Input('subs_ticker', 'value'),
Input('signal1_ticker', 'value'),
Input('sites_ticker', 'value'),
Input('signal2_ticker', 'value'),
Input('ratio_ticker', 'value')])
def update_graph(selected_subs, selected_sig1, selected_sites, selected_sig2, selected_ratios):
    fig = go.Figure()

    for x in selected_subs:
        for y in selected_sig1:
            fig.add_trace(go.Scattergl(x=customer[customer.Substation == x].datetime, y=customer[customer.Substation == x][y],
            name = str(x) + ' ' + str(y)))

    for x in selected_sites:
        for y in selected_sig2:
            fig.add_trace(go.Scattergl(x=weather[weather.Site == x].datetime, y=weather[weather.Site == x][y],
            name = str(x) + ' ' + str(y)))

    for sub in selected_subs:
        for site in selected_sites:
            for ratio in selected_ratios: # P_GEN/SolarRad
                weather_upsampled = weather[weather.Site == site].set_index('datetime').resample('10T').asfreq().reset_index()
                denom = weather_upsampled[ratio[6:]]
                num = customer[customer.Substation == sub][ratio[0:5]]
                fig.add_trace(go.Scattergl(x=customer[customer.Substation == sub].datetime, y=num.divide(denom).replace(np.inf, 0),
                name = str(sub) + ' ' + str(ratio[0:5]) + '\\' + str(site) + ' ' + str(ratio[6:])))

    fig.update_traces(
        line={"width": 0.5},
        marker={"size": 3},
        mode="lines+markers",
        showlegend=True
    )

    fig.update_layout(
        plot_bgcolor=colors['background'],
        paper_bgcolor=colors['background'],
        font_color=colors['text'],
        dragmode="zoom", template="plotly_white", legend_orientation="v", xaxis_rangeslider_visible=True,
        xaxis_rangeslider_thickness=0.1, xaxis_rangeslider_bgcolor=colors['text'],
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                    label="1d",
                    step="day",
                    stepmode="backward"),
                    dict(count=1,
                    label="1m",
                    step="month",
                    stepmode="backward"),
                    dict(step="all")
                ])
            )
        ))
    
    return fig

# Run app and display result inline in the notebook
app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:8050/
